In [ ]:
#colab file link: https://colab.research.google.com/drive/1cYyolS8q6GA1VFkqrNO-pi3z5KxrTZ0H?usp=sharing

In [ ]:
!pip install composio_crewai
!pip install crewai
!pip install langchain-google-genai
!pip install 'crewai[tools]' composio-core
!pip install composio-langchain
!pip install --upgrade stackapi
!pip install huggingface_hub
!pip install google-search-results
!pip install gradio
!pip install composio-core
!pip install langchain-community

!composio-cli add serpapi

In [ ]:
from crewai import Agent, Task, Crew, Process
from composio_langchain import ComposioToolset, Action, App
from langchain_google_genai import ChatGoogleGenerativeAI
import os
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

llm = ChatGoogleGenerativeAI(
        model="gemini-pro", verbose=True, temperature=0.9, google_api_key=os.getenv("GOOGLE_API_KEY")
    )

tools = ComposioToolset(actions=[Action.SERPAPI_SEARCH])


researcher = Agent(
  role='Investment Researcher',
  goal='Use SERP to research the top 2 results based on the input given to you and provide a report',
  backstory="""
  You are an expert Investment researcher. Using the information given to you, conduct comprehensive research using
  various sources and provide a detailed report. Dont pass in location as an argument to the tool
  """,
  verbose=True,
  allow_delegation=True,
  tools=tools,
  llm=llm
)


analyser = Agent(
  role='Investment Analyst',
  goal='Analyse the stock based on information available to it, use all the tools',
  backstory="""
  You are an expert Investment Analyst. You research on the given topic and analyse your research for insights.
  Note: Do not use SERP when youre writing the report""",
  verbose=True,
  tools=tools,
  llm=llm
)

recommend = Agent(
  role='Investment Recommendation',
  goal='Based on the analyst insights, you offer recommnedations',
  backstory="""
  You are an expert Investment Recommender. You understand the analyst insights and with your expertise suggest and offer
  advice on whether to invest or not. List the Pros and Cons as bullet points
  """,
  verbose=True,
  tools=tools,
  llm=llm
)
user_input = input(
        "Please provide a topic: "
    )


analyst_task = Task(
    description=f'Research on '+ user_input,
    agent = analyser,
    expected_output = "When the input is well researched, throughouly analysed and recommendation is offered"


)

    # Create the crew
investment_crew = Crew(
        agents=[researcher, analyser, recommend],
        tasks=[analyst_task],
        verbose=1,
        #process = Process.sequential,
        full_output = True,

    )

# Execution Flow
res = investment_crew.kickoff()